## 📖 Teoria: CIDR e Netmask

### Notazione CIDR

```
192.168.1.0/24
            ^^
            |
            Prefix length (bit di rete)

/24 = primi 24 bit sono RETE, ultimi 8 bit sono HOST
```

### Netmask

La netmask ha `prefix_len` bit a 1 (rete) seguiti da bit a 0 (host):

```
/24:
11111111.11111111.11111111.00000000
└────────────rete──────────┘└─host─┘
255.255.255.0 = 0xFFFFFF00

/16:
11111111.11111111.00000000.00000000
255.255.0.0 = 0xFFFF0000

/8:
11111111.00000000.00000000.00000000
255.0.0.0 = 0xFF000000
```

### Calcoli Fondamentali

```
Network address = IP & netmask
Broadcast address = network | ~netmask
Host count = 2^(32-prefix) - 2
```

## 🔢 Algoritmo: create_netmask()

### Da prefix_len a netmask

**Esempio: /24 → 0xFFFFFF00**

```
Step 1: Calcola host bits
  32 - 24 = 8

Step 2: Crea maschera con 8 bit a 1
  1 << 8 = 256 = 0x00000100

Step 3: Sottrai 1
  256 - 1 = 255 = 0x000000FF
  (8 bit a 1)

Step 4: Inverti con NOT
  ~0x000000FF = 0xFFFFFF00 ✓
```

### Visualizzazione Bit

```
host_bits = 8

1 << 8:
00000000 00000000 00000001 00000000
                          ^
                       bit 8 settato

(1 << 8) - 1:
00000000 00000000 00000000 11111111
                          └──8 bit─┘

~((1 << 8) - 1):
11111111 11111111 11111111 00000000
└──────────24 bit a 1────┘└─8 zero─┘
```

## 📝 Implementazione

In [ ]:
%%bash
mkdir -p /home/giordi/Repos/CUgolot/C-Struct/IPscope/tutorials/build/03
cd /home/giordi/Repos/CUgolot/C-Struct/IPscope/tutorials/build/03

# Copia file precedenti
cp ../01/ip_types.h .
cp ../02/ip_utils.* .

# Aggiungi nuove funzioni all'header
cat >> ip_utils.h << 'EOF'

// Calcoli subnet
uint32_t create_netmask(uint8_t prefix_len);
uint32_t calculate_network_address(uint32_t ip, uint32_t netmask);
uint32_t calculate_broadcast_address(uint32_t network, uint32_t netmask);
bool ip_in_subnet(uint32_t ip, uint32_t network, uint32_t netmask);
uint32_t count_available_hosts(uint8_t prefix_len);
EOF

# Implementazione
cat >> ip_utils.c << 'EOF'

/**
 * Crea netmask da lunghezza prefisso CIDR
 * 
 * Formula: ~((1 << (32 - prefix_len)) - 1)
 * 
 * Esempio /24:
 * - 32 - 24 = 8 host bits
 * - 1 << 8 = 256
 * - 256 - 1 = 255 = 0xFF
 * - ~0xFF = 0xFFFFFF00
 */
uint32_t create_netmask(uint8_t prefix_len) {
    // Casi edge
    if (prefix_len == 0) return 0x00000000;  // /0 = 0.0.0.0
    if (prefix_len >= 32) return 0xFFFFFFFF; // /32 = 255.255.255.255
    
    // Crea maschera con prefix_len bit a 1 a sinistra
    // e (32-prefix_len) bit a 0 a destra
    return ~((1U << (32 - prefix_len)) - 1);
}

/**
 * Calcola network address
 * 
 * Algoritmo: AND tra IP e netmask
 * Azzera tutti i bit della parte host
 * 
 * Esempio:
 * IP:      192.168.1.100  = 11000000.10101000.00000001.01100100
 * Netmask: 255.255.255.0  = 11111111.11111111.11111111.00000000
 *          ---------------------------------------------------
 * Network: 192.168.1.0    = 11000000.10101000.00000001.00000000
 */
uint32_t calculate_network_address(uint32_t ip, uint32_t netmask) {
    return ip & netmask;
}

/**
 * Calcola broadcast address
 * 
 * Algoritmo: OR tra network e NOT netmask
 * Imposta a 1 tutti i bit della parte host
 * 
 * Esempio /24:
 * Network:    192.168.1.0    = 11000000.10101000.00000001.00000000
 * ~Netmask:   0.0.0.255      = 00000000.00000000.00000000.11111111
 *             ---------------------------------------------------
 * Broadcast:  192.168.1.255  = 11000000.10101000.00000001.11111111
 */
uint32_t calculate_broadcast_address(uint32_t network, uint32_t netmask) {
    return network | ~netmask;
}

/**
 * Verifica se IP appartiene a subnet
 * 
 * Due IP appartengono alla stessa subnet se hanno
 * lo stesso network address dopo AND con netmask
 */
bool ip_in_subnet(uint32_t ip, uint32_t network, uint32_t netmask) {
    return (ip & netmask) == (network & netmask);
}

/**
 * Calcola host disponibili in subnet
 * 
 * Formula: 2^(32-prefix_len) - 2
 * -2 perché network e broadcast non usabili
 * 
 * Eccezioni:
 * /32: 1 host (single host route)
 * /31: 2 host (point-to-point, RFC 3021)
 */
uint32_t count_available_hosts(uint8_t prefix_len) {
    if (prefix_len > 32) return 0;
    if (prefix_len == 32) return 1;  // Single host
    if (prefix_len == 31) return 2;  // Point-to-point
    
    uint32_t host_bits = 32 - prefix_len;
    uint32_t total = 1U << host_bits;
    
    // Sottrai network e broadcast
    return total - 2;
}
EOF

echo "✅ Funzioni subnet aggiunte"

## ✅ Test Calcoli Subnet

In [ ]:
%%bash
cd /home/giordi/Repos/CUgolot/C-Struct/IPscope/tutorials/build/03

cat > test_subnet.c << 'EOF'
#include <stdio.h>
#include "ip_utils.h"

void test_netmask() {
    printf("=== TEST create_netmask() ===\n");
    
    uint8_t prefixes[] = {8, 16, 24, 28, 30, 31, 32};
    
    for (int i = 0; i < 7; i++) {
        uint8_t prefix = prefixes[i];
        uint32_t mask = create_netmask(prefix);
        char buf[16];
        ip_to_string(mask, buf);
        
        printf("  /%2d → 0x%08X → %s\n", prefix, mask, buf);
    }
    printf("\n");
}

void test_network_broadcast() {
    printf("=== TEST Network & Broadcast ===\n");
    
    // Test case: 192.168.1.100/24
    uint32_t ip;
    string_to_ip("192.168.1.100", &ip);
    uint32_t netmask = create_netmask(24);
    
    uint32_t network = calculate_network_address(ip, netmask);
    uint32_t broadcast = calculate_broadcast_address(network, netmask);
    
    char ip_str[16], net_str[16], bcast_str[16];
    ip_to_string(ip, ip_str);
    ip_to_string(network, net_str);
    ip_to_string(broadcast, bcast_str);
    
    printf("  IP:        %s\n", ip_str);
    printf("  Netmask:   /24\n");
    printf("  Network:   %s\n", net_str);
    printf("  Broadcast: %s\n\n", bcast_str);
}

void test_ip_in_subnet() {
    printf("=== TEST ip_in_subnet() ===\n");
    
    uint32_t network;
    string_to_ip("192.168.1.0", &network);
    uint32_t netmask = create_netmask(24);
    
    const char *test_ips[] = {
        "192.168.1.1",
        "192.168.1.254",
        "192.168.2.1",
        "192.168.0.255",
        "10.0.0.1"
    };
    
    printf("  Subnet: 192.168.1.0/24\n\n");
    
    for (int i = 0; i < 5; i++) {
        uint32_t ip;
        string_to_ip(test_ips[i], &ip);
        bool in_subnet = ip_in_subnet(ip, network, netmask);
        
        printf("  %-15s → %s\n", test_ips[i], 
               in_subnet ? "✓ Dentro" : "✗ Fuori");
    }
    printf("\n");
}

void test_host_count() {
    printf("=== TEST count_available_hosts() ===\n");
    
    uint8_t prefixes[] = {8, 16, 24, 28, 29, 30, 31, 32};
    
    for (int i = 0; i < 8; i++) {
        uint8_t prefix = prefixes[i];
        uint32_t hosts = count_available_hosts(prefix);
        
        printf("  /%2d → %10u host", prefix, hosts);
        
        // Mostra formula
        if (prefix < 31) {
            printf(" (2^%d - 2)", 32 - prefix);
        } else if (prefix == 31) {
            printf(" (point-to-point)");
        } else {
            printf(" (single host)");
        }
        printf("\n");
    }
    printf("\n");
}

void test_bitwise_visualization() {
    printf("=== VISUALIZZAZIONE CALCOLI ===\n");
    printf("Subnet: 192.168.1.100/24\n\n");
    
    uint32_t ip = 0xC0A80164;  // 192.168.1.100
    uint32_t mask = create_netmask(24);
    
    printf("IP:      0x%08X (192.168.1.100)\n", ip);
    printf("Netmask: 0x%08X (255.255.255.0)\n\n", mask);
    
    uint32_t network = ip & mask;
    printf("Network = IP & Netmask\n");
    printf("  0x%08X\n", ip);
    printf("& 0x%08X\n", mask);
    printf("  --------\n");
    printf("  0x%08X (192.168.1.0)\n\n", network);
    
    uint32_t broadcast = network | ~mask;
    printf("Broadcast = Network | ~Netmask\n");
    printf("  0x%08X\n", network);
    printf("| 0x%08X (~mask)\n", ~mask);
    printf("  --------\n");
    printf("  0x%08X (192.168.1.255)\n", broadcast);
}

int main() {
    test_netmask();
    test_network_broadcast();
    test_ip_in_subnet();
    test_host_count();
    test_bitwise_visualization();
    
    printf("\n✅ Tutti i test completati!\n");
    return 0;
}
EOF

gcc -std=c11 -Wall -Wextra -o test_subnet ip_utils.c test_subnet.c && ./test_subnet

## 🔍 Analisi: Perché ~((1<<n)-1)?

### Scomposizione Formula

Per /24 (24 bit rete, 8 bit host):

```
Step 1: Calcola bit host
  32 - 24 = 8

Step 2: 1 << 8
  = 1 * 2^8
  = 256
  = 0b100000000 (bit 8 settato)

Step 3: 256 - 1
  = 255
  = 0b11111111 (8 bit a 1)

Step 4: ~255
  = NOT 0x000000FF
  = 0xFFFFFF00
  = 24 bit a 1, 8 bit a 0 ✓
```

### Perché (1<<n)-1 crea n bit a 1?

```
1 << 3 = 8 = 0b1000
8 - 1  = 7 = 0b0111  ← 3 bit a 1!

1 << 4 = 16 = 0b10000
16 - 1 = 15 = 0b01111  ← 4 bit a 1!
```

**Pattern**: Sottrarre 1 da una potenza di 2 riempie tutti i bit inferiori.

## 📊 Tabella Subnet Comuni

| CIDR | Netmask | Wildcard | Host | Uso Tipico |
|------|---------|----------|------|------------|
| /8   | 255.0.0.0 | 0.255.255.255 | 16,777,214 | Classe A |
| /16  | 255.255.0.0 | 0.0.255.255 | 65,534 | Classe B |
| /24  | 255.255.255.0 | 0.0.0.255 | 254 | Classe C, LAN |
| /25  | 255.255.255.128 | 0.0.0.127 | 126 | Subnet piccola |
| /26  | 255.255.255.192 | 0.0.0.63 | 62 | VLAN |
| /27  | 255.255.255.224 | 0.0.0.31 | 30 | Subnet ufficio |
| /28  | 255.255.255.240 | 0.0.0.15 | 14 | Subnet minima |
| /29  | 255.255.255.248 | 0.0.0.7 | 6 | Tiny subnet |
| /30  | 255.255.255.252 | 0.0.0.3 | 2 | Point-to-point tradizionale |
| /31  | 255.255.255.254 | 0.0.0.1 | 2 | Point-to-point RFC 3021 |
| /32  | 255.255.255.255 | 0.0.0.0 | 1 | Single host route |

## 📚 Recap

### ✅ Funzioni implementate:

1. `create_netmask()` - /24 → 255.255.255.0
2. `calculate_network_address()` - IP & netmask
3. `calculate_broadcast_address()` - network | ~netmask
4. `ip_in_subnet()` - Verifica appartenenza
5. `count_available_hosts()` - 2^(32-prefix) - 2

### 🎓 Operazioni apprese:

- **NOT** (`~`): inversione bit
- **AND** (`&`): estrazione network
- **OR** (`|`): calcolo broadcast
- **Formula magica**: `~((1<<n)-1)`

### 🔑 Concetti chiave:

```
Netmask:   prefix_len bit a 1, resto a 0
Network:   azzera bit host (AND)
Broadcast: setta bit host a 1 (OR ~mask)
In subnet: stesso network address
```

### ➡️ Prossimo Notebook

**Notebook 4: Classificazione IP**
- `classify_ip()` - Determina classe A-E
- `get_ip_type()` - Riservato/Privato/Pubblico
- Lookup table con range speciali
- RFC 1918 e RFC 3330

---

## 🎉 Notebook 3 Completato!

**Funzioni**: 5 calcoli di subnet  
**Concetti**: CIDR, netmask, network/broadcast

Continua con il Notebook 4! 🚀